In [1]:
import numpy as np

from utils.utils import *
from utils.utils_nn import *

np.random.seed(seed())

import tensorflow as tf
tf.reset_default_graph()
tf.set_random_seed(seed())

import pandas as pd

import os

from tensorflow.contrib import rnn
from tensorflow.contrib.tensorboard.plugins import projector  # embeddings visualizer

# from collections import Counter
# from math import ceil

import random
random.seed(seed())

# import matplotlib.pyplot as plt

# import re

In [2]:
# initialize data from main (original) CSV file
x, y, n, main_data = init_data()
freq = [i for i in main_data['CNT'][:n]]  # frequencies, turned into a list
# initialize data from suggestions CSV file
x_suggest, y_suggest, freq_suggest = init_data_suggest()

In [3]:
kwargs_simple_lstm = nice_dict({
    # log
    'log_dir': 'logdir/', 
    'del_log': True, 
    # preprocessing and data
    'char_filter': 100, 
    'scale_func': unscale,  # log_scale, 
    'to_permute': True, 
    'seed': seed(), 
    # learning hyper-params
    'learn_rate': 1E-1,  # 1E-4
    'char_embed_dim': 4, 
    'one_hot': False,
    'hidden_state_size': 8, 
    'keep_prob': 0.7, 
    'epochs': 500,
    'summary_step': 5, 
    'save_step': 10
})

if kwargs_simple_lstm.del_log: remove_dir_content(kwargs_simple_lstm.log_dir)

Log directory was deleted.


In [4]:
# filter characters according to 'char_filter',
# makes all sequences the same (max) length and pads with 'unknown' character
x_char_filtered_pad, statistics_dict = \
    text_filter_pad_to_index(text=x, y=y, **kwargs_simple_lstm)
# update main dict with newly calculated figures
kwargs_simple_lstm = nice_dict({**kwargs_simple_lstm, **statistics_dict})

# create look-up dictionaries (and inverse) for an index representation
char_int, char_int_inv, label_int, label_int_inv = \
    lookup_dicts_chars_labels(**kwargs_simple_lstm)

# transform x_suggest in a similar manner
# taking into consideration the given character set
x_suggest_char_filtered_pad, statistics_dict = \
    text_filter_pad_to_index(text=x_suggest, y=y_suggest, **kwargs_simple_lstm)

# check that there are no "new" statistics popping out
assert_no_stats_change(new_dict=statistics_dict, 
                       kwargs=kwargs_simple_lstm)

# merge original and suggested data
x_merge, y_merge, freq_merge = \
    x_char_filtered_pad + x_suggest_char_filtered_pad, \
    y + y_suggest, \
    freq + freq_suggest
# y_merge = y + y_suggest
# freq_merge = freq + freq_suggest

In [5]:
# split to training and validation sets
x_val, x_train, y_val, y_train, freq_val, freq_train, valid_index = \
    train_validation_split(x=x_merge, y=y_merge, freq=freq_merge, 
                           label_count_thresh=10, 
                           valid_ratio=0.25)
n_train, n_test = len(y_train), len(y_val)

number of potential labels to draw from: 82
number of potential observation to draw from: 1587
365 observations sampled for validation
2919 total number of observations
2554 observations for training
The ratio of validation to total observations is about 0.125


In [6]:
# scale data (proportional to frequency)
# training data
x_train_scaled, y_train_scaled, kwargs_simple_lstm['n_train'] = \
    scale_permute_data(x=x_train, 
                       y=y_train, 
                       freq=freq_train, 
                       scale_func=kwargs_simple_lstm.scale_func, 
                       to_permute=kwargs_simple_lstm.to_permute)

# validation data
x_val_scaled, y_val_scaled, kwargs_simple_lstm['n_test'] = \
    scale_permute_data(x=x_val, 
                       y=y_val, 
                       freq=freq_val, 
                       scale_func=kwargs_simple_lstm.scale_func, 
                       to_permute=kwargs_simple_lstm.to_permute)

In [7]:
# aliasing, so that will run smoothly from here
x_feed_train, y_feed_train, x_feed_val, y_feed_val = \
    x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled

In [8]:
# returns np.arrays to feed into tf model
# training data
X_train, _, Y_train = index_transorm_xy(x=x_feed_train, 
                                        y=y_feed_train, 
                                        char_int=char_int, 
                                        label_int=label_int, 
                                        **kwargs_simple_lstm)

# validation data
X_val, _, Y_val = index_transorm_xy(x=x_feed_val, 
                                    y=y_feed_val, 
                                    char_int=char_int, 
                                    label_int=label_int, 
                                    **kwargs_simple_lstm)

# write a metadata file for embeddings visualizer and create path string
embed_vis_path = write_embeddings_metadata(log_dir=kwargs_simple_lstm.log_dir, 
                                           dictionary=char_int, 
                                           file_name='metadata.tsv')

In [9]:
class Lstm_model(object):

    def __init__(self, 
                 *args, 
                 hparam_str, 
                 n_train, 
                 n_test, 
                 seq_len, 
                 n_class, 
                 n_char, 
                 char_embed_dim, 
                 one_hot, 
                 hidden_state_size, 
                 keep_prob, 
                 learn_rate, 
                 epochs, 
                 log_dir, 
                 embed_vis_path, 
                 summary_step, 
                 save_step, 
                 seed, 
                 feed_dict_train, 
                 feed_dict_test, 
                 **kwargs):
#         self.feed_dict_train = {}
#         self.feed_dict_val = {}
        self.hparam_str = hparam_str
        self.n_train = n_train
        self.n_test = n_test
        self.seq_len = seq_len 
        self.n_class = n_class 
        self.n_char = n_char
        self.char_embed_dim = char_embed_dim
        self.one_hot = one_hot
        self.hidden_state_size = hidden_state_size
        self.keep_prob = keep_prob
        self.learn_rate = learn_rate
        self.epochs = epochs
        self.log_dir = log_dir
        self.embed_vis_path = embed_vis_path
        self.summary_step = summary_step 
        self.save_step = save_step
        self.seed = seed
        # placeholders
        self.embedding_matrix = None
                
        # g = tf.Graph()
        # with g.as_default():
        #     tf.set_random_seed(1)
        
#         self.g = tf.Graph()
#         self.g.seed = self.seed
    #         with self.g.as_default():
#         tf.set_random_seed(self.seed)
        self.sess = tf.Session()
        

        # Setup placeholders, and reshape the data
        self.x_ = tf.placeholder(tf.int32, [None, self.seq_len], 
                            name='Examples')
        self.y_ = tf.placeholder(tf.int32, [None, self.n_class], 
                            name='Lables')
        
        self.feed_dict_train = {self.x_: feed_dict_train['x'], 
                                self.y_: feed_dict_train['y']}
        
        self.feed_dict_test = {self.x_: feed_dict_test['x'], 
                              self.y_: feed_dict_test['y']}    

        self.embedding_matrix = self.embed_matrix()

        self.outputs = self.lstm_unit(input=self.x_)

        self.logits = self.logit(input=self.outputs, 
                            size_in=self.hidden_state_size, 
                            size_out=self.n_class)

        with tf.name_scope('cross_entropy'):
            self.cost = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(
                    logits=self.logits, labels=self.y_))
            tf.summary.scalar('cross_entropy_train', self.cost, collections=['train'])
            tf.summary.scalar('cross_entropy_test', self.cost, collections=['test'])

        with tf.name_scope('train'):
            self.train_step = tf.train.AdamOptimizer(
                self.learn_rate).minimize(self.cost)

        with tf.name_scope('accuracy'):
#             name = 'accuracy'
#             name_var = tf.Variable('accuracy', dtype=tf.string)
#             name = tf.cond(self.train_data, 
#                            lambda: self.add_train(name_var), 
#                            lambda: self.add_valid(name_var))
            self.correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.y_, 1))
            self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))
            tf.summary.scalar('accuracy_train', self.accuracy, collections=['train'])
            tf.summary.scalar('accuracy_test', self.accuracy, collections=['test'])
#             tf.summary.scalar('accuracy_validation', self.accuracy)

        # embedding vis
        self.embedding_vis = tf.Variable(tf.zeros(self.embedding_matrix.get_shape().as_list()), 
                                    trainable=False, 
                                    name='embedding_vis')
#         tf.nn.embedding_lookup(embeddings, input)
        self.assignment = self.embedding_vis.assign(self.embedding_matrix)

        # summaries per collection and saver object
#         self.summ = tf.summary.merge_all()
        
        self.summ_train = tf.summary.merge_all('train')
        self.summ_test = tf.summary.merge_all('test')
        self.saver = tf.train.Saver()

        # init vars and setup writer
        self.sess.run(tf.global_variables_initializer())
        self.writer = tf.summary.FileWriter(self.log_dir + self.hparam_str)
        self.writer.add_graph(self.sess.graph)
        
        
        # NOT SURE!
#         self.writer_val = tf.summary.FileWriter(self.log_dir + self.hparam_str + 'validation/')
#         with tf.name_scope('accuracy'):
#             tf.summary.scalar('accuracy_validation', self.accuracy, collections=['test'])
#         with tf.name_scope('cross_entropy'):
#             tf.summary.scalar('cross_entropy_validation', self.cost, collections=['test'])
#         self.summ_val = tf.summary.merge([self.accuracy, self.cost])
#         self.summ_val = tf.summary.merge(['accuracy_validation', 'cross_entropy_validation'])
        
        
        
        # Add embedding tensorboard visualization. Need tensorflow version
        self.config = projector.ProjectorConfig()
        self.embed = self.config.embeddings.add()
        self.embed.tensor_name = self.embedding_vis.name
        self.embed.metadata_path = os.path.join(self.embed_vis_path)
        projector.visualize_embeddings(self.writer, self.config)
        
        # embedding vis
        
    def embed_matrix(self, stddev=0.1, name='embeddings'):
        # index_size would be the size of the character set
        with tf.name_scope(name):
            if not self.one_hot:
                embedding_matrix = tf.get_variable(
                    'embedding_matrix', 
                    initializer=tf.truncated_normal([self.n_char, self.char_embed_dim], 
                                                    stddev=stddev, 
                                                    seed=self.seed), 
                    trainable=True)
            else:
                # creating a one-hot for each character corresponds to the identity matrix
                embedding_matrix = tf.constant(value=np.identity(self.n_char), 
                                               name='embedding_matrix', 
                                               dtype=tf.float32)
                self.char_embed_dim = self.n_char

            tf.summary.histogram('embedding_matrix', embedding_matrix, collections=['train'])
            self.embedding_matrix = embedding_matrix
            return self.embedding_matrix
        
        
    def lstm_unit(self, 
                  input, 
                  name='LSTM'):
        with tf.name_scope(name):
#             print(input.shape)  # (?, 80)
            input = tf.nn.embedding_lookup(self.embedding_matrix, input)
#             print(input.shape)  # (?, 80, 4)
            
            # reshaping
            # Permuting batch_size and n_steps
            input = tf.transpose(input, [1, 0, 2])
            # Reshaping to (n_steps*batch_size, n_input)
            input = tf.reshape(input, [-1, self.char_embed_dim])
            # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
            rnn_inputs = tf.split(input, self.seq_len, 0)

            cell = rnn.BasicLSTMCell(num_units=self.hidden_state_size)
            keep_prob = tf.constant(self.keep_prob)
            cell = rnn.DropoutWrapper(cell, 
                                      output_keep_prob=keep_prob, 
                                      seed=self.seed)

            outputs, states = rnn.static_rnn(cell, rnn_inputs, dtype=tf.float32)
            outputs = outputs[-1]
    #         outputs = tf.constant(value=outputs, 
    #                               name='outputs')
            tf.summary.histogram('outputs', outputs, collections=['train'])
            return outputs


    def logit(self, 
              input, 
              size_in, 
              size_out, 
              stddev=0.1, 
              name='logit'):

        with tf.name_scope(name):
            w = tf.Variable(tf.truncated_normal([size_in, size_out], 
                                                stddev=stddev, 
                                                seed=self.seed), 
                           name='W')
            b = tf.Variable(tf.constant(0.1, 
                                        shape=[size_out]), 
                            name='B')
            logits = tf.matmul(input, w) + b
            tf.summary.histogram('weights', w, collections=['train'])
            tf.summary.histogram('biases', b, collections=['train'])
            tf.summary.histogram('logits', logits, collections=['train'])
            return logits
    
        
    def train(self):
        print('Starting to train model {:s}'.format(self.hparam_str))
        
#         self.sess.graph.seed = self.seed
        for i in range(self.epochs + 1):
            if (i+1) % self.summary_step == 0:
                # minimizing cost (while also tracking accuracy, for summary)
                # train
                [train_accuracy, train_cost, s] = \
                    self.sess.run([self.accuracy, 
                                   self.cost, 
                                   self.summ_train],
                                  feed_dict=self.feed_dict_train)
                self.writer.add_summary(s, i)
                # test
                [test_accuracy, test_cost, s] = \
                    self.sess.run([self.accuracy, 
                                   self.cost, 
                                   self.summ_test],
                                  feed_dict=self.feed_dict_test)
                self.writer.add_summary(s, i)
                
                print('Epoch number {}, '.format(i+1) +
                      'accuracy is {:.5f} and '.format(train_accuracy) + 
                      'cost is {:.5f}'.format(train_cost))
                print('Epoch number {}, '.format(i+1) +
                      'test accuracy is {:.5f} and '.format(test_accuracy) + 
                      'test cost is {:.5f}'.format(test_cost))
                
            if (i+1) % self.save_step == 0:
                print('Saving step {}'.format(i+1))
                self.sess.run(self.assignment, feed_dict=self.feed_dict_train)
                self.saver.save(self.sess, os.path.join(self.log_dir, 
                                                        self.hparam_str, 
                                                        'model.ckpt'), i)
            self.sess.run(self.train_step, feed_dict=self.feed_dict_train)

#         self.sess.close()
        print('Training the model is done! ({:s})'.format(self.hparam_str))
    
    
    def restore(self, cp_path, feed_dict = None):
        
        print('Loading variables from {:s}'.format(cp_path))

        ckpt = tf.train.get_checkpoint_state(cp_path)
        if ckpt and ckpt.model_checkpoint_path:
            self.saver.restore(self.sess, ckpt.model_checkpoint_path)
        else:
            raise Exception("no checkpoint found")

#         if feed_dict:
#             self.feed(feed_dict=feed_dict)
        print('Loading successful')

In [10]:
# tf.reset_default_graph()

kwargs_feed_dict_train = {'x': X_train, 'y': Y_train}
kwargs_feed_dict_test = {'x': X_val, 'y': Y_val}

hparam_str = make_hparam_string(**kwargs_simple_lstm)

lstm = Lstm_model(hparam_str=hparam_str, 
                  embed_vis_path=embed_vis_path, 
                  feed_dict_train=kwargs_feed_dict_train, 
                  feed_dict_test=kwargs_feed_dict_test, 
                  **{**kwargs_simple_lstm, 
                     **{'epochs': 500}})

In [11]:
# lstm.feed_train(feed_dict_train=kwargs_feed_dict_train)
# lstm.feed_val(feed_dict_val=kwargs_feed_dict_val)
lstm.train()

Starting to train model learn_rate=1.0E-01,one_hot=F,keep_prob=0.7,char_embed_dim=4,hidden_state_size=8/
Epoch number 5, accuracy is 0.02232 and cost is 5.89566
Epoch number 5, test accuracy is 0.05479 and test cost is 5.00173
Epoch number 10, accuracy is 0.04190 and cost is 5.83024
Epoch number 10, test accuracy is 0.08767 and test cost is 5.01003
Saving step 10
Epoch number 15, accuracy is 0.03641 and cost is 5.81552
Epoch number 15, test accuracy is 0.10137 and test cost is 5.01079
Epoch number 20, accuracy is 0.04464 and cost is 5.78443
Epoch number 20, test accuracy is 0.08767 and test cost is 4.83411
Saving step 20
Epoch number 25, accuracy is 0.04581 and cost is 5.71820
Epoch number 25, test accuracy is 0.10137 and test cost is 4.83510
Epoch number 30, accuracy is 0.04464 and cost is 5.64253
Epoch number 30, test accuracy is 0.10411 and test cost is 4.77894
Saving step 30
Epoch number 35, accuracy is 0.04777 and cost is 5.55580
Epoch number 35, test accuracy is 0.09589 and test 

In [ ]:
# lstm.restore(cp_path=os.path.join(lstm.log_dir, hparam_str))

In [ ]:
# this works after training!
# need to check if it's working after calling self.restore()
# seems to work after calling self.restore(), yet numbers are not identical
# need to solve the seed() problem
# lstm.feed(feed_dict=lstm.feed_dict)
with lstm.sess.as_default() as sess:
# with lstm.sess as sess:
#     with lstm.Graph().as_default():
#         print(lstm.logits.eval(feed_dict=lstm.feed_dict))
    print(lstm.embedding_matrix.eval())

In [ ]:
# lstm.train()

In [ ]:
# trying out a LOT of hyper-parameters configurations
kwargs_feed_dict = {'x': X, 'y': Y_dense}
lstm_models = {}
for learn_rate in list(np.logspace(-1, -2, 2)):
    for keep_prob in [0.7, 1.0]:
        for one_hot, char_embed_dim in [(True, 4)] + list(zip([False] * 1 , [4])):
#             for char_embed_dim in list(np.linspace(2, 6, 3)):
            for hidden_state_size in [4, 32]:
                    current_kw_simple_lstm = {
                        **kwargs_simple_lstm, 
                        **{'learn_rate': learn_rate, 
                           'one_hot': one_hot, 
                           'keep_prob': keep_prob, 
                           'char_embed_dim': char_embed_dim, 
                           'hidden_state_size': hidden_state_size}}
                    hparam_str = make_hparam_string(learn_rate, 
                                                    one_hot, 
                                                    keep_prob, 
                                                    char_embed_dim, 
                                                    hidden_state_size)
                    var = 'lstm_{}'.format(hparam_str)
#                         print(var)
                    lstm_models[var] = Lstm_model(feed_dict=kwargs_feed_dict, 
                                                  hparam_str=hparam_str, 
                                                  embed_vis_path=embed_vis_path, 
                                                  **current_kw_simple_lstm)
                    lstm_models[var].train()


    
#     lstm_models[var] = Lstm_model(feed_dict=kwargs_feed_dict, 
#                                   hparam_str=hparam_str, 
#                                   embed_vis_path=embed_vis_path, 
#                                   **current_kw_simple_lstm)
#     lstm_models[var].train()


In [ ]:
### MNIST EMBEDDINGS ###
mnist = tf.contrib.learn.datasets.mnist.read_data_sets(train_dir=kwargs_tf_simple.log_dir + 'data', one_hot=True)
### Get a sprite and labels file for the embedding projector ###
urllib.request.urlretrieve(kwargs_tf_simple.GIST_URL + 'labels_1024.tsv', kwargs_tf_simple.log_dir + 'labels_1024.tsv')
urllib.request.urlretrieve(kwargs_tf_simple.GIST_URL + 'sprite_1024.png', kwargs_tf_simple.log_dir + 'sprite_1024.png')

pcp1()

def conv_layer(input, size_in, size_out, name="conv"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5, 5, size_in, size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        act = tf.nn.relu(conv + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")


def fc_layer(input, size_in, size_out, name="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
        act = tf.nn.relu(tf.matmul(input, w) + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act


def mnist_model(learning_rate, use_two_conv, use_two_fc, hparam):
    tf.reset_default_graph()
    sess = tf.Session()
    
    tf.set_random_seed(seed())

    # Setup placeholders, and reshape the data
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x")
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', x_image, 3)
    y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")

    if use_two_conv:
        conv1 = conv_layer(x_image, 1, 32, "conv1")
        conv_out = conv_layer(conv1, 32, 64, "conv2")
    else:
        conv1 = conv_layer(x_image, 1, 64, "conv")
        conv_out = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

    flattened = tf.reshape(conv_out, [-1, 7 * 7 * 64])


    if use_two_fc:
        fc1 = fc_layer(flattened, 7 * 7 * 64, 1024, "fc1")
        embedding_input = fc1
        embedding_size = 1024
        logits = fc_layer(fc1, 1024, 10, "fc2")
    else:
        embedding_input = flattened
        embedding_size = 7*7*64
        logits = fc_layer(flattened, 7*7*64, 10, "fc")

    with tf.name_scope("xent"):
        xent = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                logits=logits, labels=y), name="xent")
        tf.summary.scalar("xent", xent)

    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)

    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("accuracy", accuracy)

    summ = tf.summary.merge_all()


    embedding = tf.Variable(tf.zeros([1024, embedding_size]), name="test_embedding")
    assignment = embedding.assign(embedding_input)
    saver = tf.train.Saver()

    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(kwargs_tf_simple.log_dir + hparam)
    writer.add_graph(sess.graph)

    config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
    embedding_config = config.embeddings.add()
    embedding_config.tensor_name = embedding.name
    embedding_config.sprite.image_path = kwargs_tf_simple.log_dir + 'sprite_1024.png'
    embedding_config.metadata_path = kwargs_tf_simple.log_dir + 'labels_1024.tsv'
    # Specify the width and height of a single thumbnail.
    embedding_config.sprite.single_image_dim.extend([28, 28])
    tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer, config)

    pcp3()
    
    for i in range(1000 + 1):
        batch = mnist.train.next_batch(100)
        if i % 5 == 0:
            
#             pcp4()
            
            [train_accuracy, s] = sess.run([accuracy, summ], feed_dict={x: batch[0], y: batch[1]})
            writer.add_summary(s, i)
            print('Iteration number {}, Accuracy is currently {}'.format(i, train_accuracy))
        if i % 500 == 0:
            sess.run(assignment, feed_dict={x: mnist.test.images[:1024], y: mnist.test.labels[:1024]})
            saver.save(sess, os.path.join(kwargs_tf_simple.log_dir, "model.ckpt"), i)
        sess.run(train_step, feed_dict={x: batch[0], y: batch[1]})

def make_hparam_string(learning_rate, use_two_fc, use_two_conv):
    conv_param = "conv=2" if use_two_conv else "conv=1"
    fc_param = "fc=2" if use_two_fc else "fc=1"
    return "lr_%.0E,%s,%s" % (learning_rate, conv_param, fc_param)

def main():
    
#     tf.set_random_seed(seed())
    
    # You can try adding some more learning rates
    for learning_rate in [1E-4]:

        # Include "False" as a value to try different model architectures
        for use_two_fc in [False]:
            for use_two_conv in [True]:
                # Construct a hyperparameter string for each one (example: "lr_1E-3,fc=2,conv=2)
                
                pcp2()
                
                hparam = make_hparam_string(learning_rate, use_two_fc, use_two_conv)
                print('Starting run for %s' % hparam)

                # Actually run with the new settings
                mnist_model(learning_rate, use_two_fc, use_two_conv, hparam)


if __name__ == '__main__':
    main()